# Chapter 1: Initial Setup and Dependencies

In [1]:
!pip install uv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\itsme\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Chapter 2: Working with text data

In [4]:
text = '''
## Attention Is All You Need: A Paradigm Shift in Sequence Modeling

The year 2017 marked a pivotal moment in the history of deep learning with the publication of the paper "Attention Is All You Need" by Vaswani et al. from Google Brain. This seminal work introduced the **Transformer** architecture, which completely revolutionized the field of **sequence modeling**, particularly in natural language processing (NLP). Prior to the Transformer, recurrent neural networks (RNNs) and their variants like LSTMs and GRUs were the dominant architectures for tasks involving sequential data. While effective, these models suffered from inherent limitations, most notably their sequential nature, which hindered parallelization and long-range dependency modeling. The Transformer, with its reliance solely on **attention mechanisms**, elegantly sidestepped these issues, paving the way for unprecedented advancements.

### The Limitations of Recurrent Architectures

Before delving into the brilliance of the Transformer, it's crucial to understand the challenges faced by its predecessors. RNNs process sequences one element at a time, feeding the hidden state from the previous step into the current one. This sequential dependency, while seemingly intuitive for sequences, created several bottlenecks:

* **Lack of Parallelization:** Because each step depends on the previous one, RNNs cannot be fully parallelized during training. This significantly slowed down training times, especially for very long sequences.
* **Vanishing/Exploding Gradients:** While LSTMs and GRUs alleviated this to some extent, the problem of vanishing or exploding gradients still made it difficult for RNNs to capture long-range dependencies effectively. Information from early parts of a long sequence could easily be diluted or lost by the time it reached the later parts.
* **Fixed Context Window (Implicitly):** Although LSTMs have "memory cells," their ability to retain information across very long distances was still limited, making it challenging to model relationships between distant words in a sentence or across paragraphs.

### The Rise of Attention: A Glimmer of Hope

The concept of **attention** wasn't entirely new in 2017. It had already been introduced and successfully applied in conjunction with RNNs, notably in machine translation tasks. In these "encoder-decoder" RNN models with attention, the attention mechanism allowed the decoder to selectively focus on different parts of the input sequence when generating an output. This was a significant improvement, enabling the model to "attend" to relevant input tokens, rather than trying to compress all information into a single fixed-size context vector. However, attention was still an add-on, enhancing an underlying recurrent structure.

### The Radical Idea: Attention Is All You Need

The true genius of the Transformer lay in its audacious claim: **recurrent and convolutional layers were unnecessary**. The entire architecture could be built purely on attention mechanisms. This was a radical departure, as convolutions and recurrences had been the bedrock of deep learning for sequence and image processing, respectively. The paper demonstrated that attention, when properly configured, could not only match but significantly surpass the performance of existing models on challenging tasks like machine translation.

The core innovation of the Transformer is the **self-attention mechanism**, specifically **multi-head self-attention**. Instead of attending to an external sequence (as in encoder-decoder attention), self-attention allows each element in a sequence to attend to all other elements within the *same* sequence. This enables the model to weigh the importance of different words in understanding the meaning of a given word within its context. For example, in the sentence "The animal didn't cross the street because it was too tired," self-attention helps the model understand that "it" refers to "the animal."

### Deconstructing the Transformer Architecture

The Transformer architecture consists of an **encoder** and a **decoder**, each composed of multiple identical layers.

#### Encoder:
Each encoder layer has two sub-layers:
1.  **Multi-Head Self-Attention:** This is where the magic happens. It allows the model to capture dependencies between different words in the input sequence. "Multi-head" means that the attention mechanism is run multiple times in parallel, each with different learned linear projections, allowing the model to attend to information from different representation subspaces at different positions. The outputs of these attention heads are then concatenated and linearly transformed.
2.  **Feed-Forward Network:** A simple position-wise fully connected feed-forward network is applied independently to each position.

Crucially, **residual connections** and **layer normalization** are used around each of these sub-layers. Residual connections help with gradient flow and enable deeper networks, while layer normalization stabilizes training.

#### Decoder:
The decoder also has multiple identical layers, but with an additional sub-layer:
1.  **Masked Multi-Head Self-Attention:** Similar to the encoder's self-attention, but it's "masked" to prevent positions from attending to subsequent positions. This ensures that the prediction for a given position can only depend on known outputs at previous positions.
2.  **Multi-Head Encoder-Decoder Attention:** This layer allows the decoder to attend to the output of the encoder. It works similarly to the attention mechanisms used in pre-Transformer RNN-based encoder-decoder models.
3.  **Feed-Forward Network:** Same as in the encoder.

#### Positional Encoding:
Since the Transformer has no recurrence or convolution, it has no inherent notion of word order. To inject this sequential information, **positional encodings** are added to the input embeddings. These are fixed (sinusoidal functions) or learned embeddings that provide information about the absolute or relative position of each token in the sequence.

### The Impact and Legacy

The implications of "Attention Is All You Need" were profound and far-reaching:

* **Superior Performance:** Transformers quickly achieved state-of-the-art results across a wide range of NLP tasks, from machine translation and text summarization to question answering and sentiment analysis.
* **Parallelization and Scalability:** The non-sequential nature of the Transformer allowed for unprecedented parallelization during training, enabling the development of much larger and more powerful models. This was a critical factor in the rise of large pre-trained language models.
* **Foundation for Large Language Models (LLMs):** The Transformer architecture is the bedrock of virtually all modern LLMs, including BERT, GPT (GPT-2, GPT-3, GPT-4, etc.), T5, LLaMA, and many more. These models, pre-trained on massive datasets, have demonstrated remarkable capabilities in understanding and generating human language, driving the current AI revolution.
* **Transfer Learning:** The ability of Transformers to learn rich, contextualized representations made them ideal for **transfer learning**. Models pre-trained on vast amounts of text data could then be fine-tuned on smaller, task-specific datasets with excellent results, significantly reducing the need for large labeled datasets.
* **Beyond NLP:** While initially designed for NLP, the success of Transformers has extended to other domains, including computer vision (e.g., Vision Transformers for image recognition), speech processing, and even reinforcement learning. This demonstrates the versatility and generalizability of the attention mechanism.

### Challenges and Future Directions

Despite its immense success, the Transformer architecture is not without its challenges:

* **Computational Cost for Long Sequences:** The self-attention mechanism has a quadratic complexity with respect to the sequence length, meaning that as sequences get very long, the computational cost (both memory and processing time) becomes prohibitive. This is an active area of research, with efforts to develop more efficient attention mechanisms (e.g., sparse attention, linear attention, attention with local windows).
* **Interpretability:** While attention weights can sometimes offer insights into what the model is focusing on, interpreting the complex interplay of multiple attention heads and deep layers remains a significant challenge.
* **Bias:** Large language models built on Transformers can inherit and amplify biases present in their training data, leading to unfair or harmful outputs. Addressing these biases is a critical ongoing effort.
* **Energy Consumption:** Training and running these massive Transformer models require substantial computational resources and energy, raising concerns about their environmental impact.

### Conclusion

"Attention Is All You Need" didn't just introduce a new architecture; it fundamentally reshaped our understanding of how to process sequential data in deep learning. By demonstrating the power and efficiency of attention mechanisms alone, it unleashed a wave of innovation that continues to drive progress in artificial intelligence. The Transformer's elegance, scalability, and ability to capture intricate dependencies have made it the de facto standard for sequence modeling, laying the groundwork for the era of large language models and pushing the boundaries of what machines can understand and generate. The journey is far from over, with researchers continuously refining and extending the Transformer's capabilities, ensuring that attention remains at the forefront of AI research for the foreseeable future.

---'''

In [4]:
with open('sample_data.txt', 'w') as f:
    f.write(text)

In [5]:
with open("sample_data.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 9728

## Attention Is All You Need: A Paradigm Shift in Sequence Modeling

The year 2017 marked a pivota


In [6]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [7]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [8]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [9]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [10]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['##', 'Attention', 'Is', 'All', 'You', 'Need', ':', 'A', 'Paradigm', 'Shift', 'in', 'Sequence', 'Modeling', 'The', 'year', '2017', 'marked', 'a', 'pivotal', 'moment', 'in', 'the', 'history', 'of', 'deep', 'learning', 'with', 'the', 'publication', 'of']
